<a href="https://colab.research.google.com/github/austinmula/ML---Classes/blob/master/Recurrent_Neural_Networks_for_Sequences%3BSentimentAnalysiswithIMDB_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Dataset to be used - Sentiment analysis(Feeling) - binary problems
# Positive or Negative

# Recurrent Neural Networks

- Process a sequence of data (-time series or text in sentiment)
- **Recurrent** - contains loops
- output of one given layer is the input of the same layer in the next step

Time Step
-  The next point in time series 
- Next work in a sequence of words for a text sequence
- RNN takes into account the relationship among earlier and later data in a sequence 
- loops in RNNs help them to learn relationships among the data in the sequence. Good -- on its own has positive sentiment Not good has a negative sentiment,, Not is earlier in the sequence RNNs do consider the relationships among earlier and later data in the sequence. Here the words that determine sentiment are adjacent. But when considering text's meaning there can be many words to consider and an arbitrary number of words between them.

Long Short-Term Memory (LSTM)-- layer that makes the neural network recurrent.

- Text mining
- responding to Q with predicted best answers
- inter language translation
- Automated video closed captioning -speech recognition
- speech analysis




# Load the Data

The data has 25 000 training sample and 25000 testind sample. The output is either 1(posistive) and 0(negative)

In [2]:
from tensorflow.keras.datasets import imdb

- The dataset contains over 88000 unique words 
- We should then specify how much to use
- Due to system memory limitation, we will use 10000. Make use of a TPU OR GPU
- More data means you get better models

In [3]:
number_of_words = 10000

In [4]:
# Theres an issue with how tensorflow keras and numpy work

In [5]:
import numpy as np

In [6]:
np_load_old=np.load

In [7]:
# Modify the default parameters of np.load

In [8]:
np.load = lambda *a, **k: np_load_old(*a, allow_pickle=True)

In [9]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

17473536/17464789 [==============================] - 0s 0us/step


In [10]:
np.load=np_load_old

In [11]:
# Data Exploration
X_test.shape

(25000,)

In [12]:
X_train.shape

(25000,)

In [13]:
y_test.shape

(25000,)

In [14]:
y_train.shape

(25000,)

# Data Exploration

In [15]:
%pprint # toggle better printing so that elements do not display vertically

Pretty printing has been turned OFF


In [16]:
X_train[200]

[1, 14, 9, 6, 227, 196, 241, 634, 891, 234, 21, 12, 69, 6, 6, 176, 7, 4, 804, 4658, 2999, 667, 11, 12, 11, 85, 715, 6, 176, 7, 1565, 8, 1108, 10, 10, 12, 16, 1844, 2, 33, 211, 21, 69, 49, 2009, 905, 388, 99, 2, 125, 34, 6, 2, 1274, 33, 4, 130, 7, 4, 22, 15, 16, 6424, 8, 650, 1069, 14, 22, 9, 44, 4609, 153, 154, 4, 318, 302, 1051, 23, 14, 22, 122, 6, 2093, 292, 10, 10, 723, 8721, 5, 2, 9728, 71, 1344, 1576, 156, 11, 68, 251, 5, 36, 92, 4363, 133, 199, 743, 976, 354, 4, 64, 439, 9, 3059, 17, 32, 4, 2, 26, 256, 34, 2, 5, 49, 7, 98, 40, 2345, 9844, 43, 92, 168, 147, 474, 40, 8, 67, 6, 796, 97, 7, 14, 20, 19, 32, 2188, 156, 24, 18, 6090, 1007, 21, 8, 331, 97, 4, 65, 168, 5, 481, 53, 3084]

To view the original text you need to know the word to which each number corresponds to. The Keras dictionary maps words to their indices. Each word's value is its frequency. Eg. 1 is frequently occuring in that dataset

-- Training and testing data have an offset of 3. Most freq occuring word has a value of 4. (1+3). Values 0,1,2 are reserved words.

0 - padding --all the train/test must have some dimension. Some reviews may need to be handled with a 0 and some shortened.

Start of the sequence - 1 -- a token used by keras internally for learning purposes.

Unknown word - 2 -- Words that are not loaded. load_data func uses 2 for words with freq rankings > the number of words

In [17]:
word_to_index=imdb.get_word_index()

1654784/1641221 [==============================] - 0s 0us/step


In [18]:
word_to_index['great']

84

In [19]:
word_to_index['wood']

2134

In [20]:
word_to_index['the']

1

In [21]:
#lets create a mapping for checking words by frequency rating 
index_to_word={index: word for (word, index) in word_to_index.items()}

In [22]:
# We then pick the top 60 most frequently used words whose key is one
[index_to_word[i] for i in range(1, 61)]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i', 'this', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'not', 'you', 'are', 'his', 'have', 'he', 'be', 'one', 'all', 'at', 'by', 'an', 'they', 'who', 'so', 'from', 'like', 'her', 'or', 'just', 'about', "it's", 'out', 'has', 'if', 'some', 'there', 'what', 'good', 'more', 'when', 'very', 'up', 'no', 'time', 'she', 'even', 'my', 'would', 'which']

In [23]:
' '.join([index_to_word.get(i-3, '?') for i in X_train[1]])

"? big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal ? the hair is big lots of boobs ? men wear those cut ? shirts that show off their ? sickening that men actually wore them and the music is just ? trash that plays over and over again in almost every scene there is trashy music boobs and ? taking away bodies and the gym still doesn't close for ? all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then"

In [24]:
y_train[1]

0

In [25]:
# Data Preparation

In [26]:
words_per_review = 200

In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [28]:
X_train = pad_sequences(X_train, maxlen=words_per_review)

In [29]:
X_train.shape

(25000, 200)

In [30]:
X_train[0]

array([   5,   25,  100,   43,  838,  112,   50,  670,    2,    9,   35,
        480,  284,    5,  150,    4,  172,  112,  167,    2,  336,  385,
         39,    4,  172, 4536, 1111,   17,  546,   38,   13,  447,    4,
        192,   50,   16,    6,  147, 2025,   19,   14,   22,    4, 1920,
       4613,  469,    4,   22,   71,   87,   12,   16,   43,  530,   38,
         76,   15,   13, 1247,    4,   22,   17,  515,   17,   12,   16,
        626,   18,    2,    5,   62,  386,   12,    8,  316,    8,  106,
          5,    4, 2223, 5244,   16,  480,   66, 3785,   33,    4,  130,
         12,   16,   38,  619,    5,   25,  124,   51,   36,  135,   48,
         25, 1415,   33,    6,   22,   12,  215,   28,   77,   52,    5,
         14,  407,   16,   82,    2,    8,    4,  107,  117, 5952,   15,
        256,    4,    2,    7, 3766,    5,  723,   36,   71,   43,  530,
        476,   26,  400,  317,   46,    7,    4,    2, 1029,   13,  104,
         88,    4,  381,   15,  297,   98,   32, 20

In [31]:
X_train.ndim

2

In [33]:
X_test = pad_sequences(X_test, maxlen=words_per_review)

In [34]:
X_test.shape

(25000, 200)

In [35]:
# Splitting the data into validation and test data

Split the 25000 test samples into 20000 test samplea and 5000 validation . We pass the validation samples to the models fit method. validation_data argument

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_test, X_val, y_test, y_val= train_test_split(X_test, y_test, random_state=1, test_size=0.20)

In [38]:
X_test.shape


(20000, 200)

In [39]:
X_val.shape

(5000, 200)

In [40]:
from tensorflow.keras.models import Sequential

In [41]:
rnn = Sequential()

In [42]:
rnn

In [43]:
from tensorflow.keras.layers import Dense, LSTM, Embedding

# Adding and Embedding


In [44]:
rnn.add(Embedding(input_dim=number_of_words, output_dim=128, input_length=words_per_review))

In [45]:
# Adding a LSTM Layer

requireses units - which is the number of neurons in the netwrok. The more the no. of neurons in the n/w the better the models remembers.
- Length of sequences(200) - number of class we want to predict
- There are 2 classes. The ccomment is either positive or negative
- Dropout - refers to the percentage of neurons to be disabled - reduces overfitting

In [46]:
rnn.add(LSTM(units=128, dropout=0.2,reccurrent_dropout=0.2))

TypeError: ignored

In [47]:
# Add A dense Layer
# the sentiment at the end need to be a positive or a negative
# Hence we need to reduce the LSTM to one result, + or - 
# The activation function to be used is a sigmoid which is the 
# best for binary classification

In [48]:
rnn.add(Dense(units=1, activation='sigmoid'))

In [49]:
# Compile the model and Display the Summary of the model

In [50]:
rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [51]:
rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          1280000   
                                                                 
 dense (Dense)               (None, 200, 1)            129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
rnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Training and Evaluating the model

In [ ]:
results = rnn.evaluate(X_test, y_test)

In [ ]:
results